In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from seldonian import *

import numpy as np
from bounds import *
from objectives import *
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
from scipy.stats import cumfreq

import pandas as pd
from tempeh.configurations import datasets

from IPython.display import display, HTML, Markdown

from sklearn import preprocessing

%matplotlib inline


In [3]:
dataset = datasets['lawschool_passbar']()

# Load data
X_train, X_test = dataset.get_X(format=pd.DataFrame)
y_train, y_test = dataset.get_y(format=pd.Series)
A_train, A_test = dataset.get_sensitive_features(name='race', format=pd.Series)

# Combine all training data into a single data frame and glance at a few rows
all_train = pd.concat([X_train, y_train, A_train], axis=1)

display(all_train)

,lsat,ugpa,pass_bar,race
0,38.0,3.1,1,white
1,45.0,2.6,1,white
2,36.0,3.6,1,white
3,37.0,3.5,1,white
4,47.0,3.3,1,white
...,...,...,...,...
13703,36.0,3.3,1,white
13704,34.0,3.5,1,white
13705,37.0,3.5,1,white
13706,40.0,3.3,1,white


In [4]:
def compare_cdfs(data, A, num_bins=100):
    cdfs = {}
    assert len(np.unique(A)) == 2

    limits = (min(data), max(data))
    s = 0.5 * (limits[1] - limits[0]) / (num_bins - 1)
    limits = (limits[0] - s, limits[1] + s)

    for a in np.unique(A):
        subset = data[A == a]

        cdfs[a] = cumfreq(subset, numbins=num_bins, defaultreallimits=limits)

    lower_limits = [v.lowerlimit for _, v in cdfs.items()]
    bin_sizes = [v.binsize for _, v in cdfs.items()]
    actual_num_bins = [v.cumcount.size for _, v in cdfs.items()]

    assert len(np.unique(lower_limits)) == 1
    assert len(np.unique(bin_sizes)) == 1
    assert np.all([num_bins == v.cumcount.size for _, v in cdfs.items()])

    xs = lower_limits[0] + np.linspace(0, bin_sizes[0] * num_bins, num_bins)

    disparities = np.zeros(num_bins)
    for i in range(num_bins):
        cdf_values = np.clip([v.cumcount[i] / len(data[A == k]) for k, v in cdfs.items()], 0, 1)
        disparities[i] = max(cdf_values) - min(cdf_values)

    return xs, cdfs, disparities


def plot_and_compare_cdfs(data, A, num_bins=100, loc='best'):
    xs, cdfs, disparities = compare_cdfs(data, A, num_bins)

    for k, v in cdfs.items():
        plt.plot(xs, v.cumcount / len(data[A == k]), label=k)

    assert disparities.argmax().size == 1
    d_idx = disparities.argmax()

    xs_line = [xs[d_idx], xs[d_idx]]
    counts = [v.cumcount[d_idx] / len(data[A == k]) for k, v in cdfs.items()]
    ys_line = [min(counts), max(counts)]

    plt.plot(xs_line, ys_line, 'o--')
    disparity_label = "max disparity = {0:.3f}\nat {1:0.3f}".format(disparities[d_idx], xs[d_idx])
    plt.text(xs[d_idx], 1, disparity_label, ha="right", va="top")

    plt.xlabel(data.name)
    plt.ylabel("cumulative frequency")
    plt.legend(loc=loc)
    plt.show()

In [32]:
all_train_grouped = all_train.groupby('race')

counts_by_race = all_train_grouped[['lsat']].count().rename(
    columns={'lsat': 'count'})

rates_by_race = all_train_grouped[['pass_bar']].mean().rename(
    columns={'pass_bar': 'pass_bar_rate'})

summary_by_race = pd.concat([counts_by_race, rates_by_race], axis=1)
display(summary_by_race)

,count,pass_bar_rate
race,,
black,880,0.780682
white,12828,0.968428


# plot statistics from the dataset

In [6]:
# Load data
X_train, X_test = dataset.get_X(format=pd.DataFrame)
y_train, y_test = dataset.get_y(format=pd.Series)
A_train, A_test = dataset.get_sensitive_features(name='race', format=pd.Series)

# Combine all training data into a single data frame and glance at a few rows
all_train_raw = pd.concat([X_train, A_train, y_train], axis=1)
all_test_raw = pd.concat([X_test, A_test, y_test], axis=1)

all_data = pd.concat([all_train_raw, all_test_raw], axis=0)

X = all_data[['lsat', 'ugpa', 'race']]
y = all_data[['pass_bar']]
le = preprocessing.LabelEncoder()
X.loc[:,'race'] = le.fit_transform(X['race'])
scaler = preprocessing.StandardScaler()
X.loc[:,['lsat', 'ugpa']] = scaler.fit_transform(X[['lsat', 'ugpa']])
A = X['race']
A_idx = 'race'


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
all_train_grouped = all_data.groupby('race')

counts_by_race = all_train_grouped[['lsat']].count().rename(
    columns={'lsat': 'count'})

quartiles_by_race = all_train_grouped[['lsat', 'ugpa']].quantile([.25, .50, .75]).rename(
    index={0.25: "25%", 0.5: "50%", 0.75: "75%"}, level=1).unstack()

rates_by_race = all_train_grouped[['pass_bar']].mean().rename(
    columns={'pass_bar': 'pass_bar_rate'})

summary_by_race = pd.concat([counts_by_race, quartiles_by_race, rates_by_race], axis=1)
display(summary_by_race)

,count,"(lsat, 25%)","(lsat, 50%)","(lsat, 75%)","(ugpa, 25%)","(ugpa, 50%)","(ugpa, 75%)",pass_bar_rate
race,,,,,,,,
black,1346,25.0,29.0,33.0,2.6,2.9,3.2,0.778603
white,19114,34.0,38.0,41.0,3.0,3.3,3.6,0.967092


In [8]:
A_p = np.sum(A)/A.shape[0]
print(f"{A_p*100}% of the labels for 'race' is 'white'")
display(X)

93.42130987292278% of the labels for 'race' is 'white'


,lsat,ugpa,race
0,0.177166,-0.334896,1
1,1.483968,-1.543483,1
2,-0.196205,0.873692,1
3,-0.009520,0.631974,1
4,1.857340,0.148539,1
...,...,...,...
6747,1.297282,-1.301765,1
6748,-0.382891,0.148539,1
6749,-0.942949,0.631974,1
6750,-0.009520,-1.060048,1


In [9]:
# pass bar rate should be equal for race='white' or race='balck'
def create_tpr_ghat(A_idx, A_val):
    def tpr_ab(X, y_true, y_pred, delta, n=None):
        tp_a = tpr_rate(A_idx, 1)(X, y_true, y_pred)
        tp = tpr_rate(A_idx, 0)(X, y_true, y_pred)
        if method == 'ttest':
            bound = abs(ttest_bounds(tp, delta, n) - ttest_bounds(tp_a, delta, n))
        else:
            bound = abs(hoeffdings_bounds(tp, delta, n) - hoeffdings_bounds(tp_a, delta, n))
        return bound.upper - 0.3
    return tpr_ab
A_idx= 2 
tpr_ab = create_tpr_ghat(A_idx, 1)

In [10]:
#Construct the ghat
ghats = []
ghats.append({
        'fn': tpr_ab,
        'delta': 0.05
    })

method='ttest'
op_method = 'Powell'

In [11]:
estimator = LogisticRegressionSeldonianModel(X.to_numpy(), y.to_numpy().flatten(), g_hats=ghats).fit(
        opt=op_method)
print(f"Accuracy: {balanced_accuracy_score(y, estimator.predict(X))}\n")

Optimization terminated successfully.
         Current function value: 8.420856
         Iterations: 4
         Function evaluations: 522
Optimization result: Optimization terminated successfully.
Accuracy: 0.5947945200032407



## $\hat{g}$ result for the constrained optimizer

In [21]:
print(f"mean ghat value: {tpr_ab(X.to_numpy(), y.to_numpy().flatten(), estimator.predict(X), delta=0.05)}")

mean ghat value: -0.0048198973552900926


## $\hat{g}$ for constrained estimator using CMAES

In [22]:
estimator_cmaes = SeldonianAlgorithmLogRegCMAES(X.to_numpy(), y.to_numpy().flatten(), g_hats=ghats, verbose=True)
estimator_cmaes.fit()
print(f"\nAccuracy: {balanced_accuracy_score(y, estimator_cmaes.predict(X))}\n")

print(f"mean ghat value: {tpr_ab(X.to_numpy(), y.to_numpy().flatten(), estimator_cmaes.predict(X), delta=0.05)}")

Max number of iters: 1000
max iterations: 16000
Current evaluation: 10024	 average loss:0.6833955838465434 

Stopping early after no change in 1008 iterations !!

Accuracy: 0.4537854534447306

mean ghat value: -0.11565917528511899


## $\hat{g}$ for unconstrained optimizer using `scipy.optimize` package

In [23]:
uc_estimator = LogisticRegressionSeldonianModel(X.to_numpy(), y.to_numpy().flatten(), g_hats=[]).fit(
        opt=op_method)
print(f"Accuracy: {balanced_accuracy_score(y, uc_estimator.predict(X))}\n")

print(f"mean ghat value: {tpr_ab(X.to_numpy(), y.to_numpy().flatten(), uc_estimator.predict(X), delta=0.05)}")

Optimization terminated successfully.
         Current function value: 0.156320
         Iterations: 9
         Function evaluations: 434
Optimization result: Optimization terminated successfully.
Accuracy: 0.5266121708892941

mean ghat value: 0.5598497611186035


## $\hat{g}$ for unconstrained CMA-ES optimizer

In [25]:
uc_estimator_cmaes = LogisticRegressionCMAES(X.to_numpy(), y.to_numpy().flatten(), verbose=True)
uc_estimator_cmaes.fit()
print(f"Accuracy: {balanced_accuracy_score(y, uc_estimator_cmaes.predict(X))}\n")
print(f"mean ghat value: {tpr_ab(X.to_numpy(), y.to_numpy().flatten(), uc_estimator_cmaes.predict(X), delta=0.05)}")

Max number of iters: 1000
max iterations: 16000
Current evaluation: 2016	 average loss:0.15580481614806882 

Stopping early after no change in 1008 iterations !!
Accuracy: 0.5182125610347902

mean ghat value: 0.5594203561313207


# Sklearn estimator

In [27]:
logreg_sk = LogisticRegression().fit(X.to_numpy(),y.to_numpy().flatten())
print(f"Accuracy: {balanced_accuracy_score(y, uc_estimator.predict(X))}\n")
print(f"mean ghat value: {tpr_ab(X.to_numpy(), y.to_numpy().flatten(), logreg_sk.predict(X), delta=0.05)}")

Accuracy: 0.5266121708892941

mean ghat value: 0.5594203561313207
